In [13]:
!pip install -q -U google-generativeai python-dotenv

I0000 00:00:1722406206.810522 6006664 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [15]:
import google.generativeai as genai
from IPython.display import Markdown, Image

In [24]:
import os, json
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [69]:
actual_data = {
    "images/1.png": {
        "calories": 357.81,
        "protein": 20.00,
        "carbs": 20.96,
        "fat": 21.69
    },
    "images/2.png": {
        "calories": 655.99,
        "protein": 93.24,
        "carbs": 23.34,
        "fat": 20.22
    },
    "images/3.png": {
        "calories": 351.89,
        "protein": 12.10,
        "carbs": 57.50,
        "fat": 9.77
    },
    "images/4.png": {
        "calories": 184.99,
        "protein": 24.54,
        "carbs": 8.90,
        "fat": 6.14
    }
}

In [70]:
def get_nutrition_info(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    You are an expert in nutrition. 
    Given an image of a food item, estimate the total nutrition information of the food item (calories, protein, carbs, and fat), 
    Given that the image was taken 50 cm above the food item, you will need to estimate the portion size of the food item.
    Output your best answer no matter what. 
    Output the nutrition information in the following JSON format:
    {
        "calories": <calories>,
        "protein": <protein>,
        "carbs": <carbs>,
        "fat": <fat>
    }
    """
    response = model.generate_content([sample_file, prompt])

    return response.text

In [71]:
def get_percent_error(image_path):    
    actual_values = actual_data[image_path]
    predicted_values = json.loads(get_nutrition_info(image_path))
    
    output = {}
    for key in actual_values:
        actual = actual_values[key]
        predicted = predicted_values[key]
        percent_error = abs((predicted - actual) / actual) * 100
        output[key] = {
            "actual": actual,
            "predicted": predicted,
            "percent_error": round(percent_error, 2)
        }
    
    return output

In [72]:
percent_error_1 = get_percent_error("images/1.png")
percent_error_1

{'calories': {'actual': 357.81, 'predicted': 450, 'percent_error': 25.77},
 'protein': {'actual': 20.0, 'predicted': 25, 'percent_error': 25.0},
 'carbs': {'actual': 20.96, 'predicted': 40, 'percent_error': 90.84},
 'fat': {'actual': 21.69, 'predicted': 25, 'percent_error': 15.26}}

In [73]:
percent_error_2 = get_percent_error("images/2.png")
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 600, 'percent_error': 8.54},
 'protein': {'actual': 93.24, 'predicted': 60, 'percent_error': 35.65},
 'carbs': {'actual': 23.34, 'predicted': 40, 'percent_error': 71.38},
 'fat': {'actual': 20.22, 'predicted': 30, 'percent_error': 48.37}}

In [77]:
percent_error_3 = get_percent_error("images/3.png")
percent_error_3

{'calories': {'actual': 351.89, 'predicted': 400, 'percent_error': 13.67},
 'protein': {'actual': 12.1, 'predicted': 10, 'percent_error': 17.36},
 'carbs': {'actual': 57.5, 'predicted': 70, 'percent_error': 21.74},
 'fat': {'actual': 9.77, 'predicted': 10, 'percent_error': 2.35}}

In [80]:
percent_error_4 = get_percent_error("images/4.png")
percent_error_4

{'calories': {'actual': 184.99, 'predicted': 350, 'percent_error': 89.2},
 'protein': {'actual': 24.54, 'predicted': 30, 'percent_error': 22.25},
 'carbs': {'actual': 8.9, 'predicted': 25, 'percent_error': 180.9},
 'fat': {'actual': 6.14, 'predicted': 15, 'percent_error': 144.3}}

In [64]:
print(get_nutrition_info("images/5.jpeg"))

{
        "calories": 700,
        "protein": 40,
        "carbs": 90,
        "fat": 25
    }



In [67]:
print(get_nutrition_info("images/6.jpeg"))

{
    "calories": 700,
    "protein": 45,
    "carbs": 80,
    "fat": 25
}



In [68]:
print(get_nutrition_info("images/7.jpeg"))

{
    "calories": 60,
    "protein": 0,
    "carbs": 15,
    "fat": 0
}

